In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
df = pd.read_csv("../../data/Dataset3/data1.csv")

In [3]:
df

,id,country_id,league_id,season,date,home_team_goal,away_team_goal,B365H,B365D,B365A,...,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA
0,1,1,1,2008/2009,2008-08-17 00:00:00,1,1,1.73,3.40,5.00,...,4.20,1.85,3.2,3.5,1.80,3.3,3.75,NaN,NaN,NaN
1,2,1,1,2008/2009,2008-08-16 00:00:00,0,0,1.95,3.20,3.60,...,3.95,1.90,3.2,3.5,1.90,3.2,3.50,NaN,NaN,NaN
2,3,1,1,2008/2009,2008-08-16 00:00:00,0,3,2.38,3.30,2.75,...,2.55,2.60,3.1,2.3,2.50,3.2,2.50,NaN,NaN,NaN
3,4,1,1,2008/2009,2008-08-17 00:00:00,5,0,1.44,3.75,7.50,...,6.80,1.40,3.9,6.0,1.44,3.6,6.50,NaN,NaN,NaN
4,5,1,1,2008/2009,2008-08-16 00:00:00,1,3,5.00,3.50,1.65,...,1.60,4.00,3.3,1.7,4.00,3.4,1.72,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,2015-09-22 00:00:00,1,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,25976,24558,24558,2015/2016,2015-09-23 00:00:00,1,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,25977,24558,24558,2015/2016,2015-09-23 00:00:00,2,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25977,25978,24558,24558,2015/2016,2015-09-22 00:00:00,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
countries = pd.read_csv("../../data/Dataset3/country.csv")

In [5]:
countries

,country_id,country_name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


In [6]:
df = df.merge(countries, on="country_id")

## Bet365

In [7]:
df365 = df[["id", "country_name", "league_id", "season", "home_team_goal", "away_team_goal", "B365H", "B365D", "B365A"]]

In [8]:
df365.head()

,id,country_name,league_id,season,home_team_goal,away_team_goal,B365H,B365D,B365A
0,1,Belgium,1,2008/2009,1,1,1.73,3.40,5.00
1,2,Belgium,1,2008/2009,0,0,1.95,3.20,3.60
2,3,Belgium,1,2008/2009,0,3,2.38,3.30,2.75
3,4,Belgium,1,2008/2009,5,0,1.44,3.75,7.50
4,5,Belgium,1,2008/2009,1,3,5.00,3.50,1.65


In [9]:
df365.isna().sum()

id                   0
country_name         0
league_id            0
season               0
home_team_goal       0
away_team_goal       0
B365H             3387
B365D             3387
B365A             3387
dtype: int64

In [10]:
len(df365)

25979

In [11]:
df365.isna().sum() / len(df365)

id                0.000000
country_name      0.000000
league_id         0.000000
season            0.000000
home_team_goal    0.000000
away_team_goal    0.000000
B365H             0.130375
B365D             0.130375
B365A             0.130375
dtype: float64

In [12]:
def print_missing(row):
    if pd.isna(row["B365H"]):
        return row["country_name"]

In [13]:
missing_countries = df365.apply(print_missing, axis=1)

In [14]:
missing_countries.dropna().value_counts()

Poland         1920
Switzerland    1422
Belgium          22
Portugal          8
Italy             6
France            4
Netherlands       3
Spain             1
Germany           1
dtype: int64

In [15]:
df365["country_name"].value_counts()

England        3040
Spain          3040
France         3040
Italy          3017
Germany        2448
Netherlands    2448
Portugal       2052
Poland         1920
Scotland       1824
Belgium        1728
Switzerland    1422
Name: country_name, dtype: int64

In [16]:
df365 = df365.dropna(subset=["B365H"])

In [17]:
df365.head()

,id,country_name,league_id,season,home_team_goal,away_team_goal,B365H,B365D,B365A
0,1,Belgium,1,2008/2009,1,1,1.73,3.40,5.00
1,2,Belgium,1,2008/2009,0,0,1.95,3.20,3.60
2,3,Belgium,1,2008/2009,0,3,2.38,3.30,2.75
3,4,Belgium,1,2008/2009,5,0,1.44,3.75,7.50
4,5,Belgium,1,2008/2009,1,3,5.00,3.50,1.65


In [18]:
def determine_final_result(row):
    hg = int(row["home_team_goal"])
    ag = int(row["away_team_goal"])
    if hg > ag:
        row["result"] = "1"
    elif hg == ag:
        row["result"] = "x"
    else:
        row["result"] = "2"
    return row

In [19]:
df365 = df365.apply(determine_final_result, axis = 1)

In [20]:
df365.head()

,id,country_name,league_id,season,home_team_goal,away_team_goal,B365H,B365D,B365A,result
0,1,Belgium,1,2008/2009,1,1,1.73,3.40,5.00,x
1,2,Belgium,1,2008/2009,0,0,1.95,3.20,3.60,x
2,3,Belgium,1,2008/2009,0,3,2.38,3.30,2.75,2
3,4,Belgium,1,2008/2009,5,0,1.44,3.75,7.50,1
4,5,Belgium,1,2008/2009,1,3,5.00,3.50,1.65,2


In [21]:
def determine_bookie_prediction(row):
    h = row["B365H"]
    d = row["B365D"]
    a = row["B365A"]
    
    if h == min([h, d, a]):
        row["prediction"] = "1"
        row["prediction_performance"] = h
    elif d == min([h, d, a]):
        row["prediction"] = "x"
        row["prediction_performance"] = d
    else:
        row["prediction"] = "2"
        row["prediction_performance"] = a
    return row

In [22]:
df365 = df365.apply(determine_bookie_prediction, axis = 1)

In [23]:
def determine_correct_prediction(row):
    row["correct"] = (row["result"] == row["prediction"])
    return row

In [24]:
df365 = df365.apply(determine_correct_prediction, axis = 1)

In [25]:
df365.head()

,id,country_name,league_id,season,home_team_goal,away_team_goal,B365H,B365D,B365A,result,prediction,prediction_performance,correct
0,1,Belgium,1,2008/2009,1,1,1.73,3.40,5.00,x,1,1.73,False
1,2,Belgium,1,2008/2009,0,0,1.95,3.20,3.60,x,1,1.95,False
2,3,Belgium,1,2008/2009,0,3,2.38,3.30,2.75,2,1,2.38,False
3,4,Belgium,1,2008/2009,5,0,1.44,3.75,7.50,1,1,1.44,True
4,5,Belgium,1,2008/2009,1,3,5.00,3.50,1.65,2,2,1.65,True


In [26]:
grouped365 = df365.groupby(["country_name", "season"])

In [27]:
grouped365.size()

country_name  season   
Belgium       2008/2009    303
              2009/2010    210
              2010/2011    240
              2011/2012    238
              2012/2013    235
                          ... 
Spain         2011/2012    380
              2012/2013    380
              2013/2014    380
              2014/2015    380
              2015/2016    380
Length: 71, dtype: int64

In [29]:
grouped365.get_group(("Belgium", '2008/2009'))

,id,country_name,league_id,season,home_team_goal,away_team_goal,B365H,B365D,B365A,result,prediction,prediction_performance,correct
0,1,Belgium,1,2008/2009,1,1,1.73,3.40,5.00,x,1,1.73,False
1,2,Belgium,1,2008/2009,0,0,1.95,3.20,3.60,x,1,1.95,False
2,3,Belgium,1,2008/2009,0,3,2.38,3.30,2.75,2,1,2.38,False
3,4,Belgium,1,2008/2009,5,0,1.44,3.75,7.50,1,1,1.44,True
4,5,Belgium,1,2008/2009,1,3,5.00,3.50,1.65,2,2,1.65,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,302,Belgium,1,2008/2009,1,2,1.44,4.20,7.50,2,1,1.44,False
302,303,Belgium,1,2008/2009,3,0,2.80,3.30,2.50,1,2,2.50,False
303,304,Belgium,1,2008/2009,0,3,2.75,3.30,2.50,2,2,2.50,True
304,305,Belgium,1,2008/2009,2,2,2.40,3.30,2.90,x,1,2.40,False


In [30]:
test = grouped365.get_group(("Belgium", '2008/2009'))

## See percentage of accuracy per country

In [31]:
test.head()

,id,country_name,league_id,season,home_team_goal,away_team_goal,B365H,B365D,B365A,result,prediction,prediction_performance,correct
0,1,Belgium,1,2008/2009,1,1,1.73,3.40,5.00,x,1,1.73,False
1,2,Belgium,1,2008/2009,0,0,1.95,3.20,3.60,x,1,1.95,False
2,3,Belgium,1,2008/2009,0,3,2.38,3.30,2.75,2,1,2.38,False
3,4,Belgium,1,2008/2009,5,0,1.44,3.75,7.50,1,1,1.44,True
4,5,Belgium,1,2008/2009,1,3,5.00,3.50,1.65,2,2,1.65,True


In [32]:
accuracies = []
for name, group in grouped365:
    total_acc = group["correct"].sum() / len(group["correct"]) * 100
    print("Group: ", name, " accuracy: ", total_acc)
    accuracies.append((name[0], name[1], total_acc))

Group:  ('Belgium', '2008/2009')  accuracy:  53.79537953795379
Group:  ('Belgium', '2009/2010')  accuracy:  53.80952380952381
Group:  ('Belgium', '2010/2011')  accuracy:  53.75
Group:  ('Belgium', '2011/2012')  accuracy:  55.88235294117647
Group:  ('Belgium', '2012/2013')  accuracy:  48.51063829787234
Group:  ('Belgium', '2014/2015')  accuracy:  52.083333333333336
Group:  ('Belgium', '2015/2016')  accuracy:  52.083333333333336
Group:  ('England', '2008/2009')  accuracy:  53.94736842105263
Group:  ('England', '2009/2010')  accuracy:  55.52631578947368
Group:  ('England', '2010/2011')  accuracy:  50.78947368421053
Group:  ('England', '2011/2012')  accuracy:  52.10526315789473
Group:  ('England', '2012/2013')  accuracy:  53.1578947368421
Group:  ('England', '2013/2014')  accuracy:  60.0
Group:  ('England', '2014/2015')  accuracy:  53.68421052631579
Group:  ('England', '2015/2016')  accuracy:  47.368421052631575
Group:  ('France', '2008/2009')  accuracy:  49.6042216358839
Group:  ('France'

In [33]:
accuracies

[('Belgium', '2008/2009', 53.79537953795379),
 ('Belgium', '2009/2010', 53.80952380952381),
 ('Belgium', '2010/2011', 53.75),
 ('Belgium', '2011/2012', 55.88235294117647),
 ('Belgium', '2012/2013', 48.51063829787234),
 ('Belgium', '2014/2015', 52.083333333333336),
 ('Belgium', '2015/2016', 52.083333333333336),
 ('England', '2008/2009', 53.94736842105263),
 ('England', '2009/2010', 55.52631578947368),
 ('England', '2010/2011', 50.78947368421053),
 ('England', '2011/2012', 52.10526315789473),
 ('England', '2012/2013', 53.1578947368421),
 ('England', '2013/2014', 60.0),
 ('England', '2014/2015', 53.68421052631579),
 ('England', '2015/2016', 47.368421052631575),
 ('France', '2008/2009', 49.6042216358839),
 ('France', '2009/2010', 50.65963060686016),
 ('France', '2010/2011', 44.854881266490764),
 ('France', '2011/2012', 51.31578947368421),
 ('France', '2012/2013', 51.578947368421055),
 ('France', '2013/2014', 52.368421052631575),
 ('France', '2014/2015', 54.473684210526315),
 ('France', '20

In [34]:
test = pd.DataFrame(accuracies)

In [35]:
test.columns = ["country_name", "season", "accuracy_percentage"]

In [36]:
by_season = test.groupby("season")

In [37]:
by_season.get_group("2015/2016")

,country_name,season,accuracy_percentage
6,Belgium,2015/2016,52.083333
14,England,2015/2016,47.368421
22,France,2015/2016,49.340369
30,Germany,2015/2016,53.267974
38,Italy,2015/2016,53.421053
46,Netherlands,2015/2016,55.228758
54,Portugal,2015/2016,55.555556
62,Scotland,2015/2016,46.491228
70,Spain,2015/2016,54.736842


In [38]:
for name, group in by_season:
    group.to_csv("{}.csv".format(name.replace("/","_")))

### Follow the bookie strategie

In [ ]:
# def determine_gain(row):
#     amount = 10
#     if row["correct"]:
#         row["gain"] = row["prediction_performance"] * amount - amount
#     else:
#         row["gain"] = -amount
#     return row

In [ ]:
# test = test.apply(determine_gain, axis = 1)

In [ ]:
# test.head()

In [ ]:
# final_gain = test["gain"].sum()

In [ ]:
# ax = test["gain"].cumsum().plot()
# ax.set_xlabel("time")
# ax.set_ylabel("gains ($)")

In [ ]:
# for name, group in grouped365:
#     current = group.apply(determine_gain, axis = 1)
#     final_gain = current["gain"].sum()
#     current["gain"].cumsum().plot()
#     plt.xlabel("time {}".format(name))
#     plt.ylabel("gains ($)")
#     plt.show()